## **Table driven Approach**

In [ ]:
from IPython.display import display, HTML

In [ ]:
DIRECTIONS = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def initialize_game_state():
    board = [[None for _ in range(8)] for _ in range(8)]
    board[3][3], board[4][4] = 'White', 'White'
    board[3][4], board[4][3] = 'Black', 'Black'
    return board

def is_valid_move(board, row, col, player):
    if board[row][col] is not None:
        return False
    opponent = 'Black' if player == 'White' else 'White'
    valid_move = False
    for dr, dc in DIRECTIONS:
        r, c = row + dr, col + dc
        found_opponent = False
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            r, c = r + dr, c + dc
            found_opponent = True
        if found_opponent and 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            valid_move = True
    return valid_move

def has_valid_moves(board, player):
    return any(is_valid_move(board, r, c, player) for r in range(8) for c in range(8))

def flip_discs(board, row, col, player):
    opponent = 'Black' if player == 'White' else 'White'
    for dr, dc in DIRECTIONS:
        r, c = row, col
        discs_to_flip = []
        r += dr
        c += dc
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            discs_to_flip.append((r, c))
            r += dr
            c += dc
        if 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            for r, c in discs_to_flip:
                board[r][c] = player

def make_move(board, row, col, player):
    if not is_valid_move(board, row, col, player):
        return False
    board[row][col] = player
    flip_discs(board, row, col, player)
    return True

def count_discs(board):
    black_discs = sum(row.count('Black') for row in board)
    white_discs = sum(row.count('White') for row in board)
    return black_discs, white_discs

def determine_winner(board):
    black_discs, white_discs = count_discs(board)
    if black_discs > white_discs:
        return 'Black wins!'
    elif white_discs > black_discs:
        return 'White wins!'
    else:
        return 'Tie game!'

def get_player_move(player):
    valid_input = False
    while not valid_input:
        try:
            row_col = input(f"{player}'s turn. Enter your move (row,col): ")
            if row_col == "-1,-1":
                quit_game()
            elif row_col == "-2,-2":
                reset_game()
            row, col = map(int, row_col.split(','))
            if 0 <= row < 8 and 0 <= col < 8:
                valid_input = True
            else:
                print("Invalid input. Please enter values between 0 and 7.")
        except ValueError:
            print("Invalid input. Please enter the row and column separated by a comma.")
    return row, col

def quit_game():
    print("Quitting the game.")
    exit()

def reset_game():
    print("Resetting the game.")
    play_othello()

def generate_othello_board(board):
    html_content = "<table border='1' style='border-collapse: collapse;'>"
    html_content += "<tr><th></th>"
    for i in range(8):
        html_content += f"<th>{i}</th>"
    html_content += "</tr>"

    for i, row in enumerate(board):
        html_content += "<tr>"
        html_content += f"<td>{i}</td>"
        for cell in row:
            if cell is None:
                html_content += "<td style='width: 30px; height: 30px; background-color: green;'></td>"
            elif cell == 'Black':
                html_content += "<td style='width: 30px; height: 30px; background-color: black; border-radius: 50%;'></td>"
            elif cell == 'White':
                html_content += "<td style='width: 30px; height: 30px; background-color: white; border-radius: 50%;'></td>"
        html_content += "</tr>"
    html_content += "</table>"
    display(HTML(html_content))

In [ ]:
# @title Table-driven approach for percepts, actions, and moves

PERCEPTS = {
    'valid_move': lambda board, row, col, player: board[row][col] is None,
    'opponent': lambda player: 'Black' if player == 'White' else 'White',
    'found_opponent': lambda r, c, dr, dc, opponent: 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent,
    'valid_coordinates': lambda r, c: 0 <= r < 8 and 0 <= c < 8,
    'player_disc': lambda r, c, player: board[r][c] == player,
}

ACTIONS = {
    'print_board': lambda board: generate_othello_board(board),
    'quit_game': lambda: quit_game(),
    'reset_game': lambda: reset_game(),
    'flip_discs': lambda board, row, col, player: flip_discs(board, row, col, player),
}


MOVES = {
    'is_valid_move': lambda board, row, col, player: any(PERCEPTS['valid_move'](board, row, col, player) for _, _ in DIRECTIONS),
    'has_valid_moves': lambda board, player: any(MOVES['is_valid_move'](board, r, c, player) for r in range(8) for c in range(8)),

    'make_move': lambda board, row, col, player: make_move(board, row, col, player),
    'get_player_move': lambda player: get_player_move(player),
}



def play_othello():
    board = initialize_game_state()
    current_player = 'Black'
    display(generate_othello_board(board))
    game_over = False


    while True:
        if not MOVES['has_valid_moves'](board, current_player):
            print(f"No valid moves for {current_player}, turn forfeited.")
            current_player = 'Black' if current_player == 'White' else 'White'
            if not MOVES['has_valid_moves'](board, current_player):
                print("Game over.")
                winner = determine_winner(board)
                print(winner)
                break

        if current_player == 'Black':
            row, col = MOVES['get_player_move'](current_player)
            if MOVES['make_move'](board, row, col, current_player):
                ACTIONS['print_board'](board)
                current_player = 'White'
            else:
                print("Invalid move. Try again.")
        else:

            print("AI's turn or other player's turn.")
            for r in range(8):
                for c in range(8):
                    if MOVES['make_move'](board, r, c, current_player):
                        print(f"{current_player} plays at ({r},{c}).")
                        ACTIONS['print_board'](board)
                        current_player = 'Black'
                        break
                else:
                    continue
                break

play_othello()

,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


None

Black's turn. Enter your move (row,col): 4,5


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


AI's turn or other player's turn.
White plays at (3,5).


...

Black's turn. Enter your move (row,col): 7,5


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


AI's turn or other player's turn.
White plays at (7,6).


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


No valid moves for Black, turn forfeited.
Game over.
White wins!


# **predicate-logic in pattern recognition**


Yes, this implementation of Othello employs elements of intelligent agent design, specifically incorporating knowledge-based reasoning and predicate logic. Here's why:

Knowledge-Based Reasoning:

>Pattern Recognition and Recording: The AI recognizes patterns in the game state and records them to make informed decisions. It maintains a history of moves and outcomes, inferring strategies based on past experiences.

Decision Making:
> The AI evaluates possible moves based on past patterns and game history to make strategic decisions, selecting moves that have higher win rates or align with recognized patterns.

Predicate Logic:
> Controlled Position Predicate: The function controlled_position() checks if a position on the board is controlled by a player based on the rules of the game. It uses predicates and logical checks to determine this condition.

Logical Rules for Moves:
> The function infer_move() uses logical rules and conditions based on game state and previous moves to decide the best move. It applies predicate logic to assess the board and make decisions.

Overall:
> The use of history tracking, pattern recognition, and logical inference to make decisions aligns with the characteristics of intelligent agents. Predicate logic is utilized to define rules and conditions for move selection and pattern recognition. Hence, this implementation showcases elements of intelligent agent behavior in a knowledge-based and predicate logic-driven context within the game of Othello.





In [ ]:
# @title Basic functions
import math

def initialize_game_state():
    board = [['.' for _ in range(8)] for _ in range(8)]
    board[3][3], board[4][4] = 'White', 'White'
    board[3][4], board[4][3] = 'Black', 'Black'
    return board

def is_valid_move(board, row, col, player):
    if board[row][col] != '.':
        return False
    opponent = 'Black' if player == 'White' else 'White'
    directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
    valid_move = False
    for dr, dc in directions:
        r, c = row + dr, col + dc
        found_opponent = False
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            r, c = r + dr, c + dc
            found_opponent = True
        if found_opponent and 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            valid_move = True
    return valid_move

def has_valid_moves(board, player):
    return any(is_valid_move(board, r, c, player) for r in range(8) for c in range(8))

def determine_winner(board):
    black_discs, white_discs = count_discs(board)
    if black_discs > white_discs:
        return 'Black wins!'
    elif white_discs > black_discs:
        return 'White wins!'
    else:
        return 'Tie game!'


def count_discs(board):
    black_discs = sum(row.count('Black') for row in board)
    white_discs = sum(row.count('White') for row in board)
    return black_discs, white_discs

def make_move(board, row, col, player):
    if not is_valid_move(board, row, col, player):
        return False
    board[row][col] = player
    flip_discs(board, row, col, player)
    player_move_history[player].append((row, col))
    return True

def flip_discs(board, row, col, player):
    directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
    opponent = 'Black' if player == 'White' else 'White'
    for dr, dc in directions:
        r, c = row, col
        discs_to_flip = []
        r += dr
        c += dc
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            discs_to_flip.append((r, c))
            r += dr
            c += dc
        if 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            for r, c in discs_to_flip:
                board[r][c] = player

def get_player_move(player):
    valid_input = False
    while not valid_input:
        try:
            row_col = input(f"{player}'s turn. Enter your move (row,col): ")
            row, col = map(int, row_col.split(','))
            if row >= 0 and row < 8 and col >= 0 and col < 8:
                valid_input = True
            else:
                print("Invalid input. Please enter values between 0 and 7.")
        except ValueError:
            print("Invalid input. Please enter the row and column separated by a comma.")
    return row, col

def generate_othello_board(board):
    html_content = "<table border='1' style='border-collapse: collapse;'>"
    html_content += "<tr><th></th>"
    for i in range(8):
        html_content += f"<th>{i}</th>"
    html_content += "</tr>"

    for i, row in enumerate(board):
        html_content += "<tr>"
        html_content += f"<td>{i}</td>"
        for cell in row:
            if cell == '.':
                html_content += "<td style='width: 30px; height: 30px; background-color: green;'></td>"
            elif cell == 'Black':
                html_content += "<td style='width: 30px; height: 30px; background-color: black; border-radius: 50%;'></td>"
            elif cell == 'White':
                html_content += "<td style='width: 30px; height: 30px; background-color: white; border-radius: 50%;'></td>"
        html_content += "</tr>"
    html_content += "</table>"
    display(HTML(html_content))


In [ ]:
# @title Predicates

def controlled_position(board, row, col, player):
    directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
    opponent = 'Black' if player == 'White' else 'White'

    for dr, dc in directions:
        r, c = row + dr, col + dc
        found_opponent = False
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            r, c = r + dr, c + dc
            found_opponent = True
        if found_opponent and 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            return True
    return False

def corner_move(row, col):
    return (row, col) in [(0, 0), (0, 7), (7, 0), (7, 7)]

def edge_move(row, col):
    return row in [0, 7] or col in [0, 7]

def opponent_last_move(board, row, col, player):
    opponent = 'Black' if player == 'White' else 'White'
    return (row, col) == player_move_history[opponent][-1] if player_move_history[opponent] else False

In [ ]:
# @title **Logical Rules**

def infer_move(board, player, patterns, row, col):
    opponent = 'Black' if player == 'White' else 'White'

    if opponent_last_move(board, row, col, player):
        valid_moves = [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, player)]
        adjacent_moves = [(r, c) for (r, c) in valid_moves if abs(r - row) <= 1 and abs(c - col) <= 1]
        return adjacent_moves[0] if adjacent_moves else valid_moves[0]

    elif edge_move(row, col):
        valid_moves = [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, player)]
        edge_moves = [(r, c) for (r, c) in valid_moves if r in [0, 7] or c in [0, 7]]
        corner_moves = [(r, c) for (r, c) in valid_moves if corner_move(r, c)]

        if edge_moves:
            return edge_moves[0]
        elif corner_moves:
            return corner_moves[0]
        else:
            return valid_moves[0]

    else:
        valid_moves = [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, player)]
        corner_moves = [(r, c) for (r, c) in valid_moves if corner_move(r, c)]
        edge_moves = [(r, c) for (r, c) in valid_moves if r in [0, 7] or c in [0, 7]]

        if corner_moves:
            return corner_moves[0]
        elif edge_moves:
            return edge_moves[0]
        else:
            best_move = None
            best_score = -math.inf
            sorted_favorable_patterns = analyze_patterns(patterns)

            for move in valid_moves:
                hypothetical_board = [row[:] for row in board]
                make_move(hypothetical_board, move[0], move[1], player)
                hypothetical_pattern = tuple(tuple(row) for row in hypothetical_board)

                if hypothetical_pattern in sorted_favorable_patterns:
                    pattern_score = patterns[hypothetical_pattern]['win_rate']

                    if pattern_score > best_score:
                        best_score = pattern_score
                        best_move = move

            return best_move if best_move else valid_moves[0]

In [ ]:
# @title Record Patterns during Gameplay

def record_pattern(board, player, outcome):
    board_pattern = tuple(tuple(row) for row in board)

    if board_pattern not in patterns:
        patterns[board_pattern] = {
            'occurrences': 1,
            'outcome': outcome,
            'win_rate': 1 if outcome == f"{player} wins!" else 0
        }
    else:
        patterns[board_pattern]['occurrences'] += 1
        if patterns[board_pattern]['outcome'] != outcome:
            patterns[board_pattern]['outcome'] = 'Inconsistent'

        patterns[board_pattern]['win_rate'] = patterns[board_pattern]['occurrences'] / patterns[board_pattern]['occurrences'] + 1


def get_ai_move(board, player):
    opponent = 'Black' if player == 'White' else 'White'

    if player_move_history[opponent]:
        opp_last_row, opp_last_col = player_move_history[opponent][-1]

        if corner_move(opp_last_row, opp_last_col):
            valid_moves = [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, player)]
            adjacent_moves = [(r, c) for (r, c) in valid_moves if abs(r - opp_last_row) <= 1 and abs(c - opp_last_col) <= 1]
            return adjacent_moves[0] if adjacent_moves else valid_moves[0]

        elif edge_move(opp_last_row, opp_last_col):
            valid_moves = [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, player)]
            edge_moves = [(r, c) for (r, c) in valid_moves if r in [0, 7] or c in [0, 7]]
            corner_moves = [(r, c) for (r, c) in valid_moves if corner_move(r, c)]

            if corner_moves:
                return corner_moves[0]
            elif edge_moves:
                return edge_moves[0]
            else:
                return infer_move(board, player, patterns, opp_last_row, opp_last_col)

        else:
            return infer_move(board, player, patterns, opp_last_row, opp_last_col)

    else:
        valid_moves = [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, player)]
        corner_moves = [(r, c) for (r, c) in valid_moves if corner_move(r, c)]

        if corner_moves:
            return corner_moves[0]
        else:
            return valid_moves[0]


In [ ]:
# @title Othello play

patterns = {}
player_move_history = {'Black': [], 'White': []}

def play_othello():
    board = initialize_game_state()
    current_player = 'Black'
    board = initialize_game_state()
    generate_othello_board(board)

    while True:
        if not has_valid_moves(board, current_player):
            print(f"No valid moves for {current_player}, turn forfeited.")
            current_player = 'Black' if current_player == 'White' else 'White'
            if not has_valid_moves(board, current_player):
                print("Game over.")
                winner = determine_winner(board)
                print(winner)
                break

        if current_player == 'Black':
            row, col = get_player_move(current_player)
            if make_move(board, row, col, current_player):
                player_move_history[current_player].append((row, col))
                outcome = determine_winner(board)
                record_pattern(board, current_player, outcome)
                generate_othello_board(board)
                current_player = 'White'
            else:
                print("Invalid move. Try again.")
        else:
            row, col = get_ai_move(board, current_player)
            if make_move(board, row, col, current_player):
                player_move_history[current_player].append((row, col))
                outcome = determine_winner(board)
                record_pattern(board, current_player, outcome)
                print(f"{current_player} plays at ({row},{col}).")
                generate_othello_board(board)
                current_player = 'Black'
            else:
                print("Invalid move. AI forfeits turn.")

# Run the game
play_othello()

,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


Black's turn. Enter your move (row,col): 4,5


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


White plays at (3,5).


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


...

Black's turn. Enter your move (row,col): 6,6


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


White plays at (7,6).


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


No valid moves for Black, turn forfeited.
Game over.
White wins!


# **min-max in pattern recognition**

In [ ]:
# @title Minimax with alpha-beta pruning

def evaluate(board, player):
    ai_discs, opponent_discs = count_discs(board)
    disc_diff = 100 * (ai_discs - opponent_discs)

    def corner_move(row, col):
        return 100 if board[row][col] == player else (-100 if board[row][col] != '.' else 0)

    corners = 100 * (corner_move(0, 0) + corner_move(0, 7) + corner_move(7, 0) + corner_move(7, 7))

    def edge_move(row, col):
        return 10 if board[row][col] == player else (-10 if board[row][col] != '.' else 0)

    edges = 10 * (
        edge_move(0, 2) + edge_move(0, 3) + edge_move(0, 4) + edge_move(0, 5) +
        edge_move(2, 0) + edge_move(3, 0) + edge_move(4, 0) + edge_move(5, 0) +
        edge_move(7, 2) + edge_move(7, 3) + edge_move(7, 4) + edge_move(7, 5) +
        edge_move(2, 7) + edge_move(3, 7) + edge_move(4, 7) + edge_move(5, 7)
    )

    return disc_diff + corners + edges

def minimax(board, depth, player, alpha, beta):
    if depth == 0 or not has_valid_moves(board, player):
        return evaluate(board, 'Black'), None

    valid_moves = [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, player)]
    best_move = None

    if player == 'Black':
        max_val = float('-inf')
        for move in valid_moves:
            new_board = [row[:] for row in board]
            make_move(new_board, move[0], move[1], player)
            score, _ = minimax(new_board, depth - 1, 'White', alpha, beta)
            if score > max_val:
                max_val = score
                best_move = move
            alpha = max(alpha, score)
            if beta <= alpha:
                break
        return max_val, best_move
    else:
        min_val = float('inf')
        for move in valid_moves:
            new_board = [row[:] for row in board]
            make_move(new_board, move[0], move[1], player)
            score, _ = minimax(new_board, depth - 1, 'Black', alpha, beta)
            if score < min_val:
                min_val = score
                best_move = move
            beta = min(beta, score)
            if beta <= alpha:
                break
        return min_val, best_move

def get_ai_move(board, player):
    _, best_move = minimax(board, 4, player, float('-inf'), float('inf'))
    return best_move

In [ ]:
# @title Othello Play

patterns = {}
player_move_history = {'Black': [], 'White': []}

def play_othello_minmax():
    board = initialize_game_state()
    current_player = 'Black'
    board = initialize_game_state()
    generate_othello_board(board)

    while True:
        if not has_valid_moves(board, current_player):
            print(f"No valid moves for {current_player}, turn forfeited.")
            current_player = 'Black' if current_player == 'White' else 'White'
            if not has_valid_moves(board, current_player):
                print("Game over.")
                winner = determine_winner(board)
                print(winner)
                break

        if current_player == 'Black':
            row, col = get_player_move(current_player)
            if make_move(board, row, col, current_player):
                player_move_history[current_player].append((row, col))
                outcome = determine_winner(board)
                record_pattern(board, current_player, outcome)
                generate_othello_board(board)
                current_player = 'White'
            else:
                print("Invalid move. Try again.")
        else:
            row, col = get_ai_move(board, current_player)
            if make_move(board, row, col, current_player):
                player_move_history[current_player].append((row, col))
                outcome = determine_winner(board)
                record_pattern(board, current_player, outcome)
                print(f"{current_player} plays at ({row},{col}).")
                generate_othello_board(board)
                current_player = 'Black'
            else:
                print("Invalid move. AI forfeits turn.")


# Run the game
play_othello_minmax()

,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


Black's turn. Enter your move (row,col): 4,2
Invalid move. Try again.
Black's turn. Enter your move (row,col): 4,5


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


White plays at (5,3).


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


...

Black's turn. Enter your move (row,col): 7,1


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


White plays at (7,2).


,0,1,2,3,4,5,6,7
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,,,,,,,,


No valid moves for Black, turn forfeited.
Game over.
White wins!
